# Applying Machine Learning to Sentiment Analysis

In [16]:
import pandas as pd
import os
"""
labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = 'D:/ml/aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
df.columns = ['review', 'sentiment']

import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('D:/ml/movie_data.csv', index=False)
"""

In [18]:
df = pd.read_csv('D:/ml/movie_data.csv')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


# Transforming words into feature vectors

In [25]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer() # 1-gram
docs = np.array([
    'The sun is shining',
    'The weather is sweet', 
    'The sun is shining, the weather is sweet, and one and one is two'
])
bag = count.fit_transform(docs)
print(count.vocabulary_)

{u'and': 0, u'sun': 4, u'is': 1, u'two': 7, u'one': 2, u'weather': 8, u'sweet': 5, u'the': 6, u'shining': 3}


In [26]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


# Assessing word relevancy via term frequency-inverse document frequency (tf-idf)

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(bag).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


# Cleaning text data

In [29]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [31]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [32]:
preprocessor("</a>This :) is :(  a test :-)!")

'this is a test :) :( :)'

In [33]:
df['review'] = df['review'].apply(preprocessor)

# Processing documents into tokens (nltk)

In [34]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [35]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [36]:
tokenizer_porter('runners like running and thus they run')

[u'runner', 'like', u'run', 'and', u'thu', 'they', 'run']

In [37]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hhe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [39]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]

['runner', u'like', u'run', u'run', 'lot']

In [40]:
tokenizer_porter('a runner likes running and runs a lot')

['a', 'runner', u'like', u'run', 'and', u'run', 'a', 'lot']